<a href="https://colab.research.google.com/github/ancestor9/Gyeongbok-AI-study/blob/main/2025%20AI%20Tutor/DSPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DSPy 시작하기(https://dspy.ai/, https://github.com/stanfordnlp/dspy)**
- DSPy is the framework for programming—rather than prompting—language models. It allows you to iterate fast on building modular AI systems and offers algorithms for optimizing their prompts and weights, whether you're building simple classifiers, sophisticated RAG pipelines, or Agent loops.
- **DSPy(Deep-learning Systems with Python)**는 LLM(Large Language Model) 기반 애플리케이션을 구축/최적화 하기 위한 강력한 프레임워크
- DSPy의 강력한 기능을 설명하기 위해 질문 답변을 위한 RAG(검색 증강 생성) 시스템을 구축하는 실제 사례를 살펴보겠습니다.
- [DSPy를 사용한 LLM 최적화: AI 시스템 구축, 최적화 및 평가를 위한 단계별 가이드](https://www.unite.ai/ko/optimize-llm-with-dspy-a-step-by-step-guide-to-build-optimize-and-evaluate-ai-systems/)

# 1단계: 언어 모델 및 검색 모델 설정
- 첫 번째 단계에는 DSPy 내에서 언어 모델(LM)과 검색 모델(RM)을 ​​구성하는 작업이 포함됩니다.

- DSPy를 설치하려면 다음을 실행하세요.

In [3]:
%%capture
# DSPy 라이브러리 설치
!pip install dspy-ai

In [10]:
import dspy
print(dspy.__version__)
# DSPy 모듈 내부 구조 확인
print(dir(dspy))

2.6.18
['Adapter', 'AvatarOptimizer', 'BaseLM', 'BaseModule', 'BestOfN', 'BetterTogether', 'BootstrapFewShot', 'BootstrapFewShotWithOptuna', 'BootstrapFewShotWithRandomSearch', 'BootstrapFinetune', 'BootstrapRS', 'COPRO', 'ChainOfThought', 'ChainOfThoughtWithHint', 'ChatAdapter', 'ColBERTv2', 'Completions', 'Embedder', 'Ensemble', 'Evaluate', 'Example', 'History', 'Image', 'InferRules', 'InputField', 'JSONAdapter', 'KNN', 'KNNFewShot', 'LM', 'LabeledFewShot', 'MIPROv2', 'Module', 'MultiChainComparison', 'OldField', 'OldInputField', 'OldOutputField', 'OutputField', 'Parallel', 'Predict', 'Prediction', 'Program', 'ProgramOfThought', 'Provider', 'PythonInterpreter', 'ReAct', 'Refine', 'Retrieve', 'SIMBA', 'Signature', 'SignatureMeta', 'Tool', 'TrainingJob', 'TwoStepAdapter', '__author__', '__author_email__', '__builtins__', '__cached__', '__description__', '__doc__', '__file__', '__loader__', '__metadata__', '__name__', '__package__', '__path__', '__spec__', '__url__', '__version__', 'ada

In [11]:
from google.colab import userdata
your_api_key = userdata.get('openapi')

- OpenAI API 키를 Google Colab의 보안 저장소에서 가져옵니다.
dspy.LM으로 GPT-4o-mini 모델을 언어 모델로 설정합니다.
ColBERTv2를 검색 모델로 설정하여 Wikipedia 2017 초록 데이터베이스에서 관련 정보를 검색할 수 있게 합니다.
dspy.settings.configure()로 기본 언어 모델과 검색 모델을 지정합니다.

In [17]:
import dspy
import openai

# OpenAI API 키 설정
openai.api_key = your_api_key  # 실제 API 키로 교체하세요

# DSPy의 LM 클래스 확인
print(dir(dspy))

# DSPy 버전 확인
print(f"DSPy 버전: {dspy.__version__}")

# 최신 DSPy에서는 LM 모듈을 통해 OpenAI 모델에 접근
# 아래 방법 최신 DSPy의 인터페이스 사용
try:
    turbo = dspy.LM(provider='openai', model='gpt-4o-mini')
except (ImportError, AttributeError):
    print("dspy.LM을 사용할 수 없습니다.")

['Adapter', 'AvatarOptimizer', 'BaseLM', 'BaseModule', 'BestOfN', 'BetterTogether', 'BootstrapFewShot', 'BootstrapFewShotWithOptuna', 'BootstrapFewShotWithRandomSearch', 'BootstrapFinetune', 'BootstrapRS', 'COPRO', 'ChainOfThought', 'ChainOfThoughtWithHint', 'ChatAdapter', 'ColBERTv2', 'Completions', 'Embedder', 'Ensemble', 'Evaluate', 'Example', 'History', 'Image', 'InferRules', 'InputField', 'JSONAdapter', 'KNN', 'KNNFewShot', 'LM', 'LabeledFewShot', 'MIPROv2', 'Module', 'MultiChainComparison', 'OldField', 'OldInputField', 'OldOutputField', 'OutputField', 'Parallel', 'Predict', 'Prediction', 'Program', 'ProgramOfThought', 'Provider', 'PythonInterpreter', 'ReAct', 'Refine', 'Retrieve', 'SIMBA', 'Signature', 'SignatureMeta', 'Tool', 'TrainingJob', 'TwoStepAdapter', '__author__', '__author_email__', '__builtins__', '__cached__', '__description__', '__doc__', '__file__', '__loader__', '__metadata__', '__name__', '__package__', '__path__', '__spec__', '__url__', '__version__', 'adapters',

- dspy: DSPy는 LLM 파이프라인 구성에 특화된 라이브러리 (dspy-ai/dspy)
- ColBERTv2: DSPy에서 제공하는 retriever 중 하나. Dense retrieval 방식이며, ColBERT 인덱스를 사용하는 고성능 검색기
- url: 연결하려는 ColBERT 서비스의 endpoint. 여기서는 http://20.102.90.50:2017/wiki17_abstracts에 ColBERTv2 서버가 떠 있어야 함

In [18]:
# Configure the LM and RM
turbo = dspy.LM(provider='openai', model='gpt-4o-mini')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)

### 🧠 DSPy 전체 흐름 이해 (RAG 기반 질의응답 시스템)

| 단계 | 설명 |
|------|------|
| **1️⃣ Dataset 준비** | `HotPotQA`, `SQuAD`, `StrategyQA` 등 질문-응답 데이터셋을 로드함 |
| **2️⃣ Input 필드 지정** | `.with_inputs('question')`으로 어떤 필드를 LLM 입력으로 쓸지 명시 |
| **3️⃣ Signature 정의** | `class QA(dspy.Signature)`로 입력/출력 구조 정의 (e.g., question, context, answer) |
| **4️⃣ Module 정의** | `class MyQAModule(dspy.Module)`로 RAG 수행 로직 구현 (`retrieve` + `predict`) |
| **5️⃣ 학습 실행 (선택)** | `dspy.Tuner()` 또는 `dspy.Examiner()`로 Prompt 구조 최적화 또는 평가 수행 |
| **6️⃣ 추론 실행** | `result = module(question=...)`으로 실제 질문에 대한 응답 생성 |


# 2단계: 데이터 세트 로드
- 다음으로 일반적으로 다중 홉 방식으로 답변되는 복잡한 질문-답변 쌍 모음이 포함된 HotPotQA 데이터 세트를 로드합니다.
- HotPotQA 데이터셋을 로드합니다. 이 데이터셋은

>> train_size=20: 20개의 학습 예제

>> dev_size=50: 50개의 개발/검증 예제

>> test_size=0: 테스트 데이터는 사용하지 않음


학습 및 검증 데이터셋에서 'question'(질문) 필드만 입력으로 사용하도록 설정합니다.

In [19]:
from dspy.datasets import HotPotQA
# Load the dataset
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)
# Specify the 'question' field as the input
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

hotpot_qa.py:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [39]:
trainset

[Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'}),
 Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'}),
 Example({'question': 'Which of these publications was most recently published, Who Put the Bomp or Self?', 'answer': 'Self'}) (input_keys={'question'}),
 Example({'question': 'The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?', 'answer': '1950'}) (input_keys={'question'}),
 Example({'question': 'Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?', 'answer': 'Tae Kwon Do Times'}) (input_keys={'question'}),
 Example({'question': 'In what year was the club founded that played Manchester City in the 1972 FA Charity Shield', 'answer': '1874'}) (input_keys={'question'}),
 Example({'question': 'Which is taller, the E

In [40]:
devset

[Example({'question': 'Are both Cangzhou and Qionghai in the Hebei province of China?', 'answer': 'no', 'gold_titles': {'Cangzhou', 'Qionghai'}}) (input_keys={'question'}),
 Example({'question': 'Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?', 'answer': 'National Hockey League', 'gold_titles': {'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}}) (input_keys={'question'}),
 Example({'question': 'The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay Lightning of the National Hockey League (NHL)?', 'answer': 'Steve Yzerman', 'gold_titles': {'Steve Yzerman', '2006–07 Detroit Red Wings season'}}) (input_keys={'question'}),
 Example({'question': 'What river is near the Crichton Collegiate Church?', 'answer': 'the River Tyne', 'gold_titles': {'Crichton Collegiate Church', 'Crichton Castle'}}) (input_keys={'que

# 3단계: 서명 작성
- DSPy는 서명을 사용하여 모듈의 동작을 정의합니다. 이 예에서는 입력 필드(컨텍스트 및 질문)와 출력 필드(답변)를 지정하여 답변 생성 작업에 대한 서명을 정의합니다.
- DSPy의 서명(Signature)은 모듈의 입출력 형식을 정의합니다:

>> context: 관련 정보를 포함할 수 있는 입력 필드

>> question: 사용자 질문을 위한 입력 필드

>> answer: 질문에 대한 답변을 위한 출력 필드

In [21]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# 4단계: 파이프라인 구축
- RAG 파이프라인을 DSPy 모듈로 구축할 것입니다.
- 이는 하위 모듈(dspy.Retrieve 및 dspy.ChainOfThought)을 선언하는 초기화 메소드(__init__)와 응답 제어 흐름을 설명하는 전달 메소드(forward)로 구성됩니다. 이 모듈을 사용하는 질문입니다.
- RAG 파이프라인을 구성합니다:

>> __init__ 메서드에서:

>>> dspy.Retrieve: 질문과 관련된 문서 k개(기본값 3)를 검색하는 모듈
dspy.ChainOfThought(GenerateAnswer): 사고 과정을 통해 답변을 생성하는 모듈


>> forward 메서드에서:

>>> 질문을 받아 관련 문서를 검색합니다
검색된 문서와 질문을 사용하여 답변을 생성합니다
최종 예측(검색된 문서와 답변)을 반환합니다

In [24]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

# 5단계: 파이프라인 최적화
- 파이프라인이 정의되었으면 이제 DSPy의 최적화 도구를 사용하여 파이프라인을 최적화할 수 있습니다.
- 이 예에서는 훈련 세트와 검증용 지표를 기반으로 모듈에 대한 효과적인 프롬프트를 생성하고 선택하는 BootstrapFewShot 최적화 프로그램을 사용합니다.

- validate_context_and_answer 함수로 평가 지표를 정의합니다:

>> answer_EM: 예측된 답변이 정답과 정확히 일치하는지 확인

>> answer_PM: 검색된 문서에 정답이 포함되어 있는지 확인

- BootstrapFewShot을 사용하여 RAG 모듈을 최적화합니다:

>> 이 과정에서 훈련 데이터를 기반으로 효과적인 프롬프트를 생성하고 선택합니다

>> 컴파일된 결과는 향상된 성능을 갖는 RAG 모듈입니다

In [28]:
# Set OpenAI API key using environment variable
import os
os.environ['OPENAI_API_KEY'] = your_api_key

In [29]:
from dspy.teleprompt import BootstrapFewShot
# Validation metric
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up the optimizer
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
# Compile the program
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

 55%|█████▌    | 11/20 [00:19<00:16,  1.78s/it]

Bootstrapped 4 full traces after 11 examples for up to 1 rounds, amounting to 11 attempts.


# 6단계: 파이프라인 평가
- 프로그램을 컴파일한 후에는 원하는 정확성과 신뢰성을 충족하는지 확인하기 위해 개발 세트에서 성능을 평가하는 것이 필수적입니다.
- Evaluate를 사용하여 컴파일된 RAG 모듈의 성능을 평가합니다
평가 결과는 정확도 백분율로 표시됩니다 (여기서는 36.0%)
평가 과정에서 4개의 스레드를 사용하여 병렬 처리를 수행합니다

In [30]:
from dspy.evaluate import Evaluate
# Set up the evaluator
evaluate = Evaluate(devset=devset, metric=validate_context_and_answer, num_threads=4, display_progress=True, display_table=0)
# Evaluate the compiled RAG program
evaluation_result = evaluate(compiled_rag)
print(f"Evaluation Result: {evaluation_result}")

Average Metric: 18.00 / 50 (36.0%): 100%|██████████| 50/50 [00:30<00:00,  1.62it/s]

2025/04/22 03:50:18 INFO dspy.evaluate.evaluate: Average Metric: 18 / 50 (36.0%)



Evaluation Result: 36.0


# 7단계: 모델 기록 검사
- 모델의 상호 작용을 더 깊이 이해하려면 모델 기록을 검사하여 가장 최근 세대를 검토할 수 있습니다.
- 이 코드는 가장 최근의 모델 상호작용을 검사하여 모델이 어떻게 작동하는지 이해할 수 있게 해줍니다. 출력에는 시스템 메시지, 사용자 메시지, 모델 응답 등이 포함됩니다.

In [31]:
# Inspect the model's history
turbo.inspect_history(n=1)





[2025-04-22T03:50:18.067820]

System message:

Your input fields are:
1. `context` (str): may contain relevant facts
2. `question` (str)
Your output fields are:
1. `reasoning` (str)
2. `answer` (str): often between 1 and 5 words
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Answer questions with short factoid answers.


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## question ## ]]
At My Window was released by which American singer-songwriter?


Assistant message:

[[ ## reasoning ## ]]
Not supplied for this particular example. 

[[ ## answer ## ]]
John Townes Van Zandt


User message:

This is an example of the task, though some input or output fields are not supplie

# 8단계: 예측하기
- 파이프라인이 최적화되고 평가되었으므로 이제 이를 사용하여 새로운 질문에 대한 예측을 할 수 있습니다.
- 새로운 질문에 대해 컴파일된 RAG 모듈을 사용하여 예측을 수행합니다
- 질문, 생성된 답변, 검색된 문서들을 출력합니다
- 예제 질문의 경우 "U.S. National Book Award"라는 정확한 답변을 제공합니다

In [32]:
# Example question
question = "Which award did Gary Zukav's first book receive?"
# Make a prediction using the compiled RAG program
prediction = compiled_rag(question)
print(f"Question: {question}")
print(f"Answer: {prediction.answer}")
print(f"Retrieved Contexts: {prediction.context}")


Question: Which award did Gary Zukav's first book receive?
Answer: U.S. National Book Award
Retrieved Contexts: ['Gary Zukav | Gary Zukav (born October 17, 1942) is an American spiritual teacher and the author of four consecutive New York Times Best Sellers. Beginning in 1998, he appeared more than 30 times on "The Oprah Winfrey Show" to discuss transformation in human consciousness concepts presented in his book "The Seat of the Soul". His first book, "The Dancing Wu Li Masters" (1979), won a U.S. National Book Award.', 'The Dancing Wu Li Masters | The Dancing Wu Li Masters is a 1979 book by Gary Zukav, a popular science work exploring modern physics, and quantum phenomena in particular. It was awarded a 1980 U.S. National Book Award in category of Science. Although it explores empirical topics in modern physics research, "The Dancing Wu Li Masters" gained attention for leveraging metaphors taken from eastern spiritual movements, in particular the Huayen school of Buddhism with the mo

In [34]:
# Example question
question = "대한민국에서 가장 유명한 고궁은? 3개만 소개해줘"
# Make a prediction using the compiled RAG program
prediction = compiled_rag(question)
print(f"Question: {question}")
print(f"Answer: {prediction.answer}")
print(f"Retrieved Contexts: {prediction.context}")

Question: 대한민국에서 가장 유명한 고궁은? 3개만 요약해서 한글로 소개해줘
Answer: 경복궁, 창덕궁, 덕수궁
Retrieved Contexts: ['June 29 Declaration | The June 29 Declaration (Korean: 6.29 선언 , Hanja: 六二九宣言, Revised Romanization: "Yug-igu seoneon"), officially titled the Special Declaration for Grand National Harmony and Progress Towards a Great Nation (Korean: 국민들의 민주화와 직선제 개헌요구를 받아들여 발표한 특별선언 ), was a speech by Roh Tae-woo, presidential candidate of the ruling Democratic Justice Party of South Korea, on 29 June 1987. In the declaration, Roh promised significant concessions to opponents of the incumbent authoritarian regime of Chun Doo-hwan who had been pressing for democracy. Roh went on to win the open presidential elections that were held that year, the first for at least the fifteen years since the October Yushin of 1972.', 'Embassy of North Korea, London | The Embassy of the Democratic People\'s Republic of Korea in London (Chosŏn\'gŭl: 주 영국 조선민주주의인민공화국 대사관 ; MR: "Chu Yŏngguk Chosŏn Minjujuŭi Inmin Konghwaguk Daesagw

## 요약
이 노트북은 DSPy를 사용하여 RAG(검색 증강 생성) 시스템을 구축하는 전체 과정을 보여줍니다:

필요한 모델 및 데이터 설정
서명과 모듈을 사용한 파이프라인 정의
최적화 및 평가를 통한 성능 향상
실제 질문에 대한 예측 수행

DSPy의 강점은 모듈화된 접근 방식과 최적화 기능으로, 개발자가 더 효과적인 AI 시스템을 신속하게 구축할 수 있게 해줍니다.